In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
from io import StringIO

with open('cleaned_data.csv', 'r', encoding="utf8") as csvfile:
    data = csvfile.read()


df = pd.read_csv(StringIO(data), header=None)
df.columns = ['team1', 'score1', 'score2', 'team2', 'date']

#df['outcome'] = np.where((df['score1'] + df['score2']) > 3.5, 1, 0)
df['score_total'] = df['score1'] + df['score2']
df['outcome'] = df['score_total'] / df['score_total'].max()
#df['outcome'] = np.where(df['score_total_normalized'] > 0.001, 1, 0)




def encode_teams(df):
    teams = pd.concat([df['team1'], df['team2']]).unique()
    team_dict = {team: i for i, team in enumerate(teams)}
    df['team1_encoded'] = df['team1'].apply(lambda x: team_dict[x])
    df['team2_encoded'] = df['team2'].apply(lambda x: team_dict[x])
    return df


df = encode_teams(df)

X = df[['team1_encoded', 'team2_encoded']]
y = df['outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(Dense(12, input_dim=2, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32)

#y_pred = np.round(model.predict(X_test)).astype(int)
y_pred = model.predict(X_test)
#accuracy = accuracy_score(y_test, y_pred)

#print("Accuracy on test set: {:.2f}%".format(accuracy * 100))

#conf_mat = confusion_matrix(y_test, y_pred)

#print("Confusion Matrix:")
#print(conf_mat)

team_dict = {i: team for team, i in encode_teams(df)[['team1', 'team1_encoded']].drop_duplicates().values}




#total_accuracy = accuracy_score(y, np.round(model.predict(X)).astype(int))
#print("Accuracy on full dataset: {:.2f}%".format(total_accuracy * 100))


Epoch 1/10
2990/2990 [==============================] - 7s 2ms/step - loss: 1.9594 - accuracy: 0.1205
Epoch 2/10
2990/2990 [==============================] - 5s 2ms/step - loss: 0.5512 - accuracy: 0.1261
Epoch 3/10
2990/2990 [==============================] - 6s 2ms/step - loss: 0.5478 - accuracy: 0.1261
Epoch 4/10
2990/2990 [==============================] - 6s 2ms/step - loss: 0.5522 - accuracy: 0.1261
Epoch 5/10
2990/2990 [==============================] - 6s 2ms/step - loss: 0.5478 - accuracy: 0.1262
Epoch 6/10
2990/2990 [==============================] - 6s 2ms/step - loss: 0.5475 - accuracy: 0.1263
Epoch 7/10
2990/2990 [==============================] - 6s 2ms/step - loss: 0.5428 - accuracy: 0.1265
Epoch 8/10
2990/2990 [==============================] - 6s 2ms/step - loss: 0.5413 - accuracy: 0.1268
Epoch 9/10
2990/2990 [==============================] - 6s 2ms/step - loss: 0.5430 - accuracy: 0.1266
Epoch 10/10
748/748 [==============================] - 1s 1ms/step


In [13]:
wins = 0
losses = 0
money = 10
target = 11
lowest = 10
highest = 10
for i in range(len(y_test)):
    team1_encoded = X_test.iloc[i]['team1_encoded']
    team2_encoded = X_test.iloc[i]['team2_encoded']
    try:
        team1 = team_dict[team1_encoded]
    except KeyError:
        team1 = 'Unknown team'
    try:
        team2 = team_dict[team2_encoded]
    except KeyError:
        team2 = 'Unknown team'
    actual = y_test.iloc[i]
    predicted = y_pred[i][0]

    single_goal = 0.0833333333333333
    predicted = predicted/single_goal
    pInt = int(round(predicted, 0))

    actual = actual/single_goal
    aInt = int(actual)


    if pInt > 7:
        if aInt > 7:
            wins += 1
            money = target
            target = money + 1
            if money > highest:
                highest = money
        else:
            losses += 1
            diff = target - money
            diffCost = diff / 50
            money -= diffCost
            if money < lowest:
                lowest = money
    

        print("Matchup: {} vs. {}. Actual Outcome: {}, Predicted Outcome: {}, Money: {}".format(team1, team2, aInt, pInt, money))
print("wins",wins,"losses", losses)
print("highest", highest, "lowest", lowest)

Matchup: Aubagne vs. Unknown team. Actual Outcome: 4, Predicted Outcome: 10, Money: 9.98
Matchup: Atlanta United vs. FC Cincinnati. Actual Outcome: 4, Predicted Outcome: 8, Money: 9.9596
Matchup: Tottenham Hotspur Women vs. London City Lionesses. Actual Outcome: 5, Predicted Outcome: 9, Money: 9.938792
Matchup: Saffron Walden Town vs. Unknown team. Actual Outcome: 6, Predicted Outcome: 9, Money: 9.91756784
Matchup: Atlanta United vs. FC Cincinnati. Actual Outcome: 2, Predicted Outcome: 8, Money: 9.8959191968
Matchup: LASK vs. WSG Tirol. Actual Outcome: 6, Predicted Outcome: 8, Money: 9.873837580736
Matchup: Racing Rioja vs. Unknown team. Actual Outcome: 3, Predicted Outcome: 9, Money: 9.85131433235072
Matchup: Racing Club Warwick vs. Coventry Sphinx. Actual Outcome: 4, Predicted Outcome: 8, Money: 9.828340618997734
Matchup: Portimonense vs. FC Famalicão. Actual Outcome: 3, Predicted Outcome: 8, Money: 9.80490743137769
Matchup: Paris Saint-Germain Féminines vs. BK Hacken Women. Actual O